In [879]:
import torch
import os, glob
import numpy as np
import random
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import f1_score
import logomaker
import matplotlib.pyplot as plt
import pandas as pd

## Creating and training the Network

In [420]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [387]:
data_pos=[]
path='/home/ubuntu/tmp/data/pos/'
for filename in glob.glob(os.path.join(path,'*.txt')):
    with open(os.path.join(os.getcwd(),filename),'r') as file:
        while line := file.readline():
            data_pos.append(line)   

data_neg=[]
path='/home/ubuntu/tmp/data/neg/'
for filename in glob.glob(os.path.join(path,'*.txt')):
    with open(os.path.join(os.getcwd(),filename),'r') as file:
        while line := file.readline():
            data_neg.append(line)

In [388]:
# Usually the number of negative data points will be more than the positive data point, since positive data is created
# by applying filter to the raw data. Whereas negative data points are also created by applying filter to the
# raw data, but the conditions are less stringent. This logic may change in the future.

total_rows=len(data_neg)+len(data_pos)
merge=[[[] for x in range(2)] for y in range(0,total_rows)]
index=0
diff=abs(len(data_neg)-len(data_pos))
for i in range(0,len(data_pos)):
    merge[index][0]=data_pos[i].strip().upper()
    merge[index][1]=1.0
    index=index+1
    merge[index][0]=data_neg[i].strip().upper()
    merge[index][1]=0.0
    index=index+1
    if diff >= 1:
        merge[index][0]=data_neg[i].strip().upper()
        merge[index][1]=0.0
        index=index+1
        diff = diff -1


In [389]:
merge[5565][0]

'AGGAAAACGTGAATCTATGTGGACTGTTCCAAACAATCCCAATTCCCCAGCTAATGAGCTCAAAGCTTTGGAAACAGGGAAAATGTCAAAGGATCCCGATTCGCCAGCTAATGAGCTGAAAGGCAATGAACCAGGAGAAGTGTCAAAAGA'

In [390]:
len(data_neg)+len(data_pos)

13190

In [391]:
# All the nucleotide are converted into a upper case letter. There are instance where the sequence may contain both upper and lower letter.
# Lower and Upper case has different interpretation, lower may be "soft mask" not completely align to the region. Lower case also means that
# it may be a repetative statement. In case of gene sequence it may mean introns and Exons (Upper). 
# If N/n is there then it will be assigned a value of [0.,0.,0.,0.]

def dna_to_one_hot_encoding(genome):
    one_hot=list()
    nucli_map={"A":[1., 0., 0., 0.], "C": [0., 1., 0., 0.], "G": [0., 0., 1., 0.], "T":[0., 0., 0., 1.]}
    #print(genome)
    for nucleotide in genome:
        one_hot.append(nucli_map[nucleotide]  if nucleotide in nucli_map.keys() else [0., 0., 0., 0.])

    return np.array(one_hot).T

In [392]:
temp=dna_to_one_hot_encoding(merge[13189][0])
temp

array([[1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
        1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        1., 1., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1.,
        0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0.,
        0., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0.,
        0., 1., 0., 0., 0., 0.,

In [393]:
# Shuffle all the indexes

index_shuffle=list(range(total_rows))
random.shuffle(index_shuffle)

train_percentage=0.7
train_split=int(len(index_shuffle)*train_percentage)
remaining=total_rows-train_split
validation_split=int(remaining*0.7)

train_idx=index_shuffle[:train_split]
validation_idx=index_shuffle[train_split:train_split+validation_split]
test_idx=index_shuffle[train_split+validation_split:]

In [394]:
# collect all the sequences and its respective label from the indexs what is already calculated for the three different splits 

train_data_raw=[]
for i in train_idx:
    train_data_raw.append(merge[i])

validation_data_raw=[]
for i in validation_idx:
    validation_data_raw.append(merge[i])

test_data_raw=[]
for i in test_idx:
    test_data_raw.append(merge[i])

In [395]:
# The custom Dataset class. The input to the function genome is a 2D data set, where the 0th dimension of an element is the sequence and
# 1st dimension is the label of the sequence. That is why x[0] is used, which contain the sequence part and x[1] contain the label. 

class genome_to_one_hot(Dataset):
    def __init__(self,genome):
        self.dna=genome

        self.genome_one_hot=torch.stack([torch.tensor(dna_to_one_hot_encoding(x[0])) for x in self.dna])
        self.labels=torch.tensor(list(x[1] for x in self.dna))

    def __len__(self):
        return len(self.dna)
    
    def __getitem__(self,id):
        one_hot_dna=self.genome_one_hot[id]
        labels=self.labels[id]

        return one_hot_dna,labels

In [396]:
# create the dataset

test_data=genome_to_one_hot(test_data_raw)
train_data=genome_to_one_hot(train_data_raw)
validation_data=genome_to_one_hot(validation_data_raw)

train_dataloader=DataLoader(train_data,batch_size=512,shuffle=True)
validation_dataloader=DataLoader(validation_data,batch_size=512,shuffle=True)
test_dataloader=DataLoader(test_data,batch_size=512)

In [397]:
class deep_peak_cnn(nn.Module):
    def __init__(self,num_classes=2):
        super().__init__()
        self.conv1=nn.Conv1d(in_channels=4,out_channels=32,kernel_size=11,stride=1,padding=0)
        self.pool1=nn.MaxPool1d(kernel_size=3,stride=2)
        self.conv2=nn.Conv1d(in_channels=32,out_channels=64,kernel_size=5,stride=1,padding=0)
        self.pool2=nn.MaxPool1d(kernel_size=3,stride=2)
        self.conv3=nn.Conv1d(in_channels=64,out_channels=128,kernel_size=3,stride=1,padding=0)
        self.pool3=nn.MaxPool1d(kernel_size=3,stride=2)
        self.conv4=nn.Conv1d(in_channels=128,out_channels=128,kernel_size=3,stride=1,padding=0)
        self.pool4=nn.MaxPool1d(kernel_size=3,stride=2)

        self.fc1=nn.Linear(in_features=(128*5),out_features=2048)
        self.fc2=nn.Linear(in_features=2048,out_features=2048)
        self.fc3=nn.Linear(in_features=2048,out_features=num_classes)

    def forward(self,x):
        x=self.pool1(F.relu(self.conv1(x)))
        x=self.pool2(F.relu(self.conv2(x)))
        x=self.pool3(F.relu(self.conv3(x)))
        x=self.pool4(F.relu(self.conv4(x)))

        x=torch.flatten(x,1)
        x=F.relu(self.fc1(x))
        x=F.dropout(x,0.6)
        x=F.relu(self.fc2(x))
        x=F.dropout(x,0.6)
        x=F.relu(self.fc3(x))

        return x


In [416]:
model=deep_peak_cnn().to(device)
optimizer=optim.Adam(params=model.parameters(),lr=0.0001)
loss_fn=nn.CrossEntropyLoss()

In [417]:
# convert the data to float

def train(model,train_loader,optimizer,epochs,device):
    model.train()
    for batch_ids, (data,labels) in enumerate(train_loader):
        labels=labels.type(torch.LongTensor)
        data,labels=data.to(device, dtype=torch.float32),labels.to(device)

        optimizer.zero_grad()
        model_output=model(data)
        loss=loss_fn(model_output,labels)
        loss.backward()
        optimizer.step()

        if (batch_ids+1)%6 == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                epochs,batch_ids*len(data),len(train_loader.dataset),
                100.*batch_ids/len(train_loader),loss.item()))


In [418]:
# f1_score is also calculated. 

def validate(model, validation_loader,device):
    model.eval()
    correct=0
    with torch.no_grad():
        for data,labels in validation_loader:
            data,labels=data.to(device,dtype=torch.float32), labels.to(device,dtype=torch.float32)
            y_hat=model(data)
            _,y_pred=torch.max(y_hat,1)
            correct+=(y_pred==labels).sum().item()
            #print("f1_score:", f1_score(labels.cpu().data,y_pred.cpu()))
        print("\n Test Set: Average loss: xx , Accuracy:{}/{} ({:.0f}%, f1_score:{})".format(
            correct,len(validation_data),100.*correct/len(validation_data),f1_score(labels.cpu().data,y_pred.cpu())))
        print("="*50)

In [419]:
if __name__=='__main__':
    seed=42
    Epochs=100

    for epoch in range(1,Epochs+1):
        train(model,train_dataloader,optimizer,epoch,device)
        validate(model,validation_dataloader,device)

Train Epoch: 1 [2560/9233 (26%)]	Loss: 0.683108
Train Epoch: 1 [5632/9233 (58%)]	Loss: 0.681863
Train Epoch: 1 [8704/9233 (89%)]	Loss: 0.673304

 Test Set: Average loss: xx , Accuracy:1582/2769 (57%, f1_score:0.0)
Train Epoch: 2 [2560/9233 (26%)]	Loss: 0.674464
Train Epoch: 2 [5632/9233 (58%)]	Loss: 0.687068
Train Epoch: 2 [8704/9233 (89%)]	Loss: 0.673895

 Test Set: Average loss: xx , Accuracy:1582/2769 (57%, f1_score:0.0)
Train Epoch: 3 [2560/9233 (26%)]	Loss: 0.664211
Train Epoch: 3 [5632/9233 (58%)]	Loss: 0.667384
Train Epoch: 3 [8704/9233 (89%)]	Loss: 0.662929

 Test Set: Average loss: xx , Accuracy:1582/2769 (57%, f1_score:0.0)
Train Epoch: 4 [2560/9233 (26%)]	Loss: 0.676531
Train Epoch: 4 [5632/9233 (58%)]	Loss: 0.680842
Train Epoch: 4 [8704/9233 (89%)]	Loss: 0.665258

 Test Set: Average loss: xx , Accuracy:1582/2769 (57%, f1_score:0.0)
Train Epoch: 5 [2560/9233 (26%)]	Loss: 0.669793
Train Epoch: 5 [5632/9233 (58%)]	Loss: 0.658925
Train Epoch: 5 [8704/9233 (89%)]	Loss: 0.662172
